In [16]:
import pandas as pd
import sklearn as skl
import matplotlib.pyplot as plt
import numpy as np
from numpy import linalg as LA

In [90]:
# 1. Загрузите данные из файла data-logistic.csv. 
# Это двумерная выборка, целевая переменная на которой принимает значения -1 или 1.

names = ['y','x1', 'x2']
data = pd.read_csv('data-logistic.csv', names = names)

x = data[['x1', 'x2']]
y = np.array(data.y)

# 2. Убедитесь, что выше выписаны правильные формулы для градиентного спуска. 
# Обратите внимание, что мы используем полноценный градиентный спуск, а не его стохастический вариант!

# 3. Реализуйте градиентный спуск для обычной и L2-регуляризованной (с коэффициентом регуляризации 10) логистической регрессии. 
# Используйте длину шага k=0.1. 
# В качестве начального приближения используйте вектор (0, 0).

# 4. Запустите градиентный спуск и доведите до сходимости 
# (евклидово расстояние между векторами весов на соседних итерациях должно быть не больше 1e-5). 
# Рекомендуется ограничить сверху число итераций десятью тысячами.

N_max = 1e5
epsilon = 1e-5
l = data.shape[0]
c = 10
k = 0.1

def GD(x, y, w0, df1, df2, N, k):
    global N_max, epsilon, l, c
    w = np.array([w0[0] - k*df1(x, y, w0, k), w0[1]- k*df2(x, y, w0, k)])
    N+=1
    if ((LA.norm(w - w0)<=epsilon)or(N==N_max)): 
        print(w, N)
        return w
    else: GD(x, y, w, df1, df2, N, k)
    

w0 = np.zeros(2)

def df1(x, y, w, k): 
    global l, c
    return -(k/l)*np.sum(y*x['x1']*(1 - 1/(1 + np.exp(-y*(w[0]*x['x1'] + w[1]*x['x2']))))) + c*w[0]

def df2(x, y, w, k): 
    global l, c
    return -(k/l)*np.sum(y*x['x2']*(1 - 1/(1 + np.exp(-y*(w[0]*x['x1'] + w[1]*x['x2']))))) + c*w[1]

def dF1(x, y, w, k): 
    global l, c
    return -(k/l)*np.sum(y*x['x1']*(1 - 1/(1 + np.exp(-y*(w[0]*x['x1'] + w[1]*x['x2']))))) 

def dF2(x, y, w, k): 
    global l, c
    return -(k/l)*np.sum(y*x['x2']*(1 - 1/(1 + np.exp(-y*(w[0]*x['x1'] + w[1]*x['x2']))))) 

w = GD(x, y, w0, df1, df2, 0, k)
W = GD(x, y, w0, dF1, dF2, 0, k)

print('FUCK UUUUUU' ,w, W)

[0.00348228 0.00314794] 3
[0.28502418 0.09456427] 1479
FUCK UUUUUU None None


In [91]:
w = np.array([0.00348228, 0.00314794])
W = np.array([0.28502418, 0.09456427])

# 5. Какое значение принимает AUC-ROC на обучении без регуляризации и при ее использовании? 
# Эти величины будут ответом на задание. 
# В качестве ответа приведите два числа через пробел. 
# Обратите внимание, что на вход функции roc_auc_score нужно подавать оценки вероятностей, подсчитанные обученным алгоритмом. 
# Для этого воспользуйтесь сигмоидной функцией: a(x) = 1/(1 + exp(−w1x1 − w2x2)).

from sklearn.metrics import roc_auc_score

def A(x, w):
    return 1/(1+np.exp(-w[0]*x['x1'] - w[1]*x['x2']))

a_without = A(x, W)
a = A(x, w)

print('check check ', roc_auc_score(y, y))
print('without regularization', roc_auc_score(y, a_without))
print('regularization', roc_auc_score(y, a))

check check  1.0
without regularization 0.927142857142857
regularization 0.9362857142857142


In [107]:
# 6. Попробуйте поменять длину шага. 
# Будет ли сходиться алгоритм, если делать более длинные шаги? 
# Как меняется число итераций при уменьшении длины шага?

print('increase')
I = np.arange(1.0, 1.81, 0.1)
for i in I:
    ww = GD(x, y, w0, df1, df2, 0, 0.1*i)

print('decrease')
for i in range(1, 6):
    ww = GD(x, y, w0, df1, df2, 0, 10**(-i))

increase
[0.00348228 0.00314794] 3
[0.00382089 0.00345236] 5
[0.00415679 0.00375391] 6
[0.00449179 0.00405444] 8
[0.00482442 0.00435244] 10
[0.0051564  0.00464968] 14
[0.00549086 0.00494959] 19
[0.0058193  0.00524312] 31
[0.00614056 0.00552872] 66
decrease
[0.00348228 0.00314794] 3
[0.00029065 0.00026396] 16
[3.57312664e-07 3.24599713e-07] 1
[3.57312664e-09 3.24599713e-09] 1
[3.57312664e-11 3.24599713e-11] 1


In [113]:
# 7. Попробуйте менять начальное приближение. 
# Влияет ли оно на что-нибудь?

for i in range(5):
    wi = np.random.rand(2)
    print(wi)
    ww = GD(x, y, wi, df1, df2, 0, k)
    print('\n')

[0.89373832 0.43693593]
[0.00348234 0.00314801] 4


[0.03861464 0.93171588]
[0.00348233 0.00314799] 4


[0.52731215 0.01369563]
[0.00348229 0.00314796] 4


[0.37465838 0.10890629]
[0.00348229 0.00314795] 4


[0.81092693 0.18818305]
[0.00348233 0.00314799] 4




In [114]:
print(np.pi)

3.141592653589793
